![ibm-cloud.png](attachment:ibm-cloud.png)

## CASE STUDY - Deploying a recommender

For this lab we will be using the MovieLens data :

* [MovieLens Downloads](https://grouplens.org/datasets/movielens/latest/)

download either **ml-latest-small.zip** or **ml-latest.zip** from this link and add the unziped folder to the data folder of the lab directory. We recommend you to use the small version if you are not working with a Spark cluster or a High memory machine.

The two important pages for documentation are below.

* [Spark MLlib collaborative filtering docs](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html) 
* [Spark ALS docs](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS)


In [2]:
pip install pyspark

     |████████████████████████████████| 204.2MB 73kB/s 
     |████████████████████████████████| 204kB 54.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=004484aae8c2bfb8d9c5840564d4f59158fa60a68670eb3e4e9163cec664f008
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest.zip 

--2021-02-04 04:19:29--  http://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277113433 (264M) [application/zip]
Saving to: ‘ml-latest.zip’

ml-latest.zip       100%[===================>] 264.28M   126MB/s    in 2.1s    

2021-02-04 04:19:31 (126 MB/s) - ‘ml-latest.zip’ saved [277113433/277113433]



In [4]:
!unzip ml-latest.zip 

Archive:  ml-latest.zip
   creating: ml-latest/
  inflating: ml-latest/links.csv     
  inflating: ml-latest/tags.csv      
  inflating: ml-latest/genome-tags.csv  
  inflating: ml-latest/ratings.csv   
  inflating: ml-latest/README.txt    
  inflating: ml-latest/genome-scores.csv  
  inflating: ml-latest/movies.csv    


In [21]:
import os
import shutil
import pandas as pd
import numpy as np
import pyspark as ps
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.sql.types import DoubleType

DATA_DIR = os.path.join(".", "/content/data/")
SAVE_DIR = os.path.join(".", "saved-recommender/")

if os.path.isdir(SAVE_DIR):
    shutil.rmtree(SAVE_DIR)

In [23]:
## ensure the spark context is available
spark = (ps.sql.SparkSession.builder
        .appName("sandbox")
        .getOrCreate()
        )

sc = spark.sparkContext
print(spark.version) 

3.0.1


### Ensure the data are downloaded, unziped and placed in the data folder of this lab.

The data can be downloaded <a href="https://grouplens.org/datasets/movielens/">here</a>. We recommend you to download the small version: <b>ml-latest-small.zip</b>

In [24]:
movielens_data_dir = os.path.join(DATA_DIR, "ml-latest")   
print(movielens_data_dir) 
if not os.path.exists(movielens_data_dir):
    print("ERROR make sure the path to the Movie Lens data is correct")

/content/data/ml-latest


In [25]:
## load the ratings data as a pysaprk dataframe
ratings_file = os.path.join(movielens_data_dir, "ratings.csv")
df = spark.read.format("csv").options(header="true", inferSchema="true").load(ratings_file)
df = df.withColumnRenamed("movieID", "movie_id")
df = df.withColumnRenamed("userID", "user_id")
df.show(n=4)

+-------+--------+------+----------+
|user_id|movie_id|rating| timestamp|
+-------+--------+------+----------+
|      1|     307|   3.5|1256677221|
|      1|     481|   3.5|1256677456|
|      1|    1091|   1.5|1256677471|
|      1|    1257|   4.5|1256677460|
+-------+--------+------+----------+
only showing top 4 rows



In [26]:
## load the movies data as a pyspark dataframe
movies_file = os.path.join(movielens_data_dir, "movies.csv") 
movies_df = spark.read.format("csv").options(header="true", inferSchema="true").load(movies_file)
movies_df = movies_df.withColumnRenamed("movieID", "movie_id")
movies_df.show(n=4)


+--------+--------------------+--------------------+
|movie_id|               title|              genres|
+--------+--------------------+--------------------+
|       1|    Toy Story (1995)|Adventure|Animati...|
|       2|      Jumanji (1995)|Adventure|Childre...|
|       3|Grumpier Old Men ...|      Comedy|Romance|
|       4|Waiting to Exhale...|Comedy|Drama|Romance|
+--------+--------------------+--------------------+
only showing top 4 rows



## QUESTION 1

Explore the movie lens data a little and summarize it.

In [27]:
## YOUR CODE HERE (summarize the data)
df.describe().show()

+-------+------------------+-----------------+------------------+--------------------+
|summary|           user_id|         movie_id|            rating|           timestamp|
+-------+------------------+-----------------+------------------+--------------------+
|  count|          27753444|         27753444|          27753444|            27753444|
|   mean|141942.01557064414|18487.99983414671|3.5304452124932677|1.1931218549319255E9|
| stddev| 81707.40009148984| 35102.6252474677|1.0663527502319696|2.1604822852233613E8|
|    min|                 1|                1|               0.5|           789652004|
|    max|            283228|           193886|               5.0|          1537945149|
+-------+------------------+-----------------+------------------+--------------------+



## QUESTION 2

Find the ten most popular movies. 


1. Create 2 pyspark dataframes one with the count of each film in df and one with the average rating of each movie in df.
2. Join these two dataframes in a third dataframe. Then, filter this dataframe to select only the movies that have been seen more than 100 times.
3. Use the movies_df dataframe to add the names of each movies on the dataframe created in 2. Then, order the dataframe by descending average rating.



In [ ]:
## YOUR CODE HERE (Replace the symbole #<> with your code)

## 1_
movie_counts = #<>
top_rated = #<>

## 2_
top_movies = #<>

## 3_
top_movies = #<>


top_movies.show(10)

+--------+-----------------+-----+--------------------+--------------------+
|movie_id|      avg(rating)|count|               title|              genres|
+--------+-----------------+-----+--------------------+--------------------+
|     318|4.429022082018927|  317|Shawshank Redempt...|         Crime|Drama|
|     858|        4.2890625|  192|Godfather, The (1...|         Crime|Drama|
|    2959|4.272935779816514|  218|   Fight Club (1999)|Action|Crime|Dram...|
|    1221| 4.25968992248062|  129|Godfather: Part I...|         Crime|Drama|
|   48516|4.252336448598131|  107|Departed, The (2006)|Crime|Drama|Thriller|
|    1213|             4.25|  126|   Goodfellas (1990)|         Crime|Drama|
|   58559|4.238255033557047|  149|Dark Knight, The ...|Action|Crime|Dram...|
|      50|4.237745098039215|  204|Usual Suspects, T...|Crime|Mystery|Thr...|
|    1197|4.232394366197183|  142|Princess Bride, T...|Action|Adventure|...|
|     260|4.231075697211155|  251|Star Wars: Episod...|Action|Adventure|...|

## QUESTION 3

We will now fit a ALS model, this is matrix factorization model used for rating recommendation. See the [Spark ALS docs](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS)
for example usage. 

First we split the data

In [28]:
(training, test) = df.randomSplit([0.8, 0.2])

Create a function called **train_model()** that takes two inputs :

1. ``reg_param`` : the regularization parameter of the factorization model
2. ``implicit_prefs`` : a boolean variable that indicate whereas the model should used explicit or implicit ratings.
    
The function train an ALS model on the training set then predict the test set and evaluate this prediction.
The output of the function should be the RMSE of the fitted model on the test set./

In [29]:
## YOUR CODE HERE (Replace the symbole #<> with your code)
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS


def train_model(reg_param, implicit_prefs=False):
    """
    Train and evaluate an ALS model
    Inputs : the regularization parametre of the ALS model and the implicit_prefs flag
    Ouptus : a string with the RMSE and the regularization parameter inputed
    """
    
    als = ALS(regParam= reg_param, userCol="user_id", itemCol="movie_id", ratingCol="rating",
          coldStartStrategy="drop", implicitPrefs  = implicit_prefs)
    model = als.fit(training)

    predictions = model.transform(test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

    rmse = evaluator.evaluate(predictions)
    print("regParam={}, RMSE={}".format(reg_param, np.round(rmse, 2)))


Calling the function created above for several ``reg_param`` values find the best regularization parameter.

In [30]:
for reg_param in [0.01, 0.05]:#, 0.1, 0.15, 0.25]:
    train_model(reg_param)

regParam=0.01, RMSE=0.83
regParam=0.05, RMSE=0.81


## QUESTION 4

With your best regParam try using the `implicitPrefs` flag.

>Note that the results here make sense because the data are `explicit` ratings

In [31]:
## YOUR CODE HERE
als = ALS(regParam= 0.05, userCol="user_id", itemCol="movie_id", ratingCol="rating",
          coldStartStrategy="drop", implicitPrefs  = True)
model = als.fit(training)

predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                            predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("regParam={}, RMSE={}".format(reg_param, np.round(rmse, 2)))

regParam=0.05, RMSE=3.25


## QUESTION 5

Retrain the model with your best ``reg_param`` and ``implicit_prefs`` on the entire dataset and save the trained model in the SAVE_DIR directory.

In [35]:
## YOUR CODE HERE (Replace the symbole #<> with your code)

### re-train using the whole data set
#print("...training")
#als = #<>
#model = als.fit(#<>)
    
## save model
print("...saving als model")
model.save(SAVE_DIR + '/model_1')
print("done.")

...saving als model
done.


## QUESTION 6

We now want to use ``spark-submit`` to load the model and demonstrate that you can load the model and interface with it.

Following the best practices we created a python script (``recommender-submit.py``) in the **scripts** folder that loads the model, creates some hand crafted data points and query the model. We recommend you to go over this script and make sure you understand it before running it through this notebook.

In [37]:
! python /content/scripts/recommender-submit.py

21/02/04 04:50:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/02/04 04:50:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
best rated [(260,), (2628,), (1196,), (122886,), (187595,), (179819,), (1210,)]
21/02/04 04:51:04 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/02/04 04:51:04 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
closest_users
 [(115850,), (148571,), (254372,), (207998,), (12897,), (82874,), (94182,), (127938,), (184509,), (21094,), (219345,), (57584,), (94098,), (117097,), (220230,), (26676,), (156690,), (154161,), (146096,), (177221,), (117290,), (254590,), (25736